In [1]:
from pyspark import SparkConf,SparkContext
from pyspark import rdd
conf=SparkConf().setMaster("local[*]").setAppName("First_APP")
sc=SparkContext(conf=conf)

----------------------
2 从数据中抽取合适的特征
----------------------



MLlib中的分类模型通过LabeledPoint对象操作,其中封装了目标变量(标签)和特征向量:

                LabeledPoint(label: Double, features: Vector) 




虽然在使用分类模型的很多样例中会碰到向量格式的数据集,但在实际工作中,通常还需要从原始数据中抽取特征 ，包括封装数值特征、归一或者正则化特征,以及使用1-of-k编码表示类属特征 


In [2]:
rawData = sc.textFile('E:/machine_data/StumbleUpon evergreen/train.tsv')
records = rawData.map(lambda x: x.split('\t'))
records=sc.parallelize(records.collect()[1:])
records.first()

['"http://www.bloomberg.com/news/2010-12-23/ibm-predicts-holographic-calls-air-breathing-batteries-by-2015.html"',
 '"4042"',
 '"{""title"":""IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries"",""body"":""A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose California Photographer Tony Avelar Bloomberg By 2015 your mobile phone will project a 3 D image of anyone who calls and your laptop will be powered by kinetic energy At least that s what International Business Machines Corp sees in its crystal ball The predictions are part of an annual tradition for the Armonk New York based company which surveys its 3 000 researchers to find five ideas expected to take root in the next five years IBM the world s l

In [3]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors
trimmed = records.map(lambda x: [xx.replace('\"','') for xx in x])
label = trimmed.map(lambda x : x[-1])

data1 = trimmed.map(lambda x : (x[-1],x[4:-1]))
print (data1.first())
data2=data1.map(lambda kv:(kv[0].replace('\"',''),[0.0 if yy=='\"?\"' else yy.replace('\"','') for yy in kv[1]]))
print (data2.first())
data3=data2.map(lambda kv:(kv[0].replace("\"",""),[0.0 if yy =='?' else yy.replace("\"","") for yy in kv[1]]))
print (data3.first())
data4=data3.map(lambda kv:(int(kv[0]), [float(yy) for yy in kv[1]]))
print (data4.first())
data=data4.map(lambda kv:LabeledPoint(kv[0],Vectors.dense(kv[1])))
data.take(5)

('0', ['0.789131', '2.055555556', '0.676470588', '0.205882353', '0.047058824', '0.023529412', '0.443783175', '0', '0', '0.09077381', '0', '0.245831182', '0.003883495', '1', '1', '24', '0', '5424', '170', '8', '0.152941176', '0.079129575'])
('0', ['0.789131', '2.055555556', '0.676470588', '0.205882353', '0.047058824', '0.023529412', '0.443783175', '0', '0', '0.09077381', '0', '0.245831182', '0.003883495', '1', '1', '24', '0', '5424', '170', '8', '0.152941176', '0.079129575'])
('0', ['0.789131', '2.055555556', '0.676470588', '0.205882353', '0.047058824', '0.023529412', '0.443783175', '0', '0', '0.09077381', '0', '0.245831182', '0.003883495', '1', '1', '24', '0', '5424', '170', '8', '0.152941176', '0.079129575'])
(0, [0.789131, 2.055555556, 0.676470588, 0.205882353, 0.047058824, 0.023529412, 0.443783175, 0.0, 0.0, 0.09077381, 0.0, 0.245831182, 0.003883495, 1.0, 1.0, 24.0, 0.0, 5424.0, 170.0, 8.0, 0.152941176, 0.079129575])


[LabeledPoint(0.0, [0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575]),
 LabeledPoint(1.0, [0.574147,3.677966102,0.50802139,0.288770053,0.213903743,0.144385027,0.468648998,0.0,0.0,0.098707403,0.0,0.203489628,0.088652482,1.0,1.0,40.0,0.0,4973.0,187.0,9.0,0.181818182,0.125448029]),
 LabeledPoint(1.0, [0.996526,2.382882883,0.562015504,0.321705426,0.120155039,0.042635659,0.525448029,0.0,0.0,0.072447859,0.0,0.22640177,0.120535714,1.0,1.0,55.0,0.0,2240.0,258.0,11.0,0.166666667,0.057613169]),
 LabeledPoint(1.0, [0.801248,1.543103448,0.4,0.1,0.016666667,0.0,0.480724749,0.0,0.0,0.095860566,0.0,0.265655744,0.035343035,1.0,0.0,24.0,0.0,2737.0,120.0,5.0,0.041666667,0.100858369]),
 LabeledPoint(0.0, [0.719157,2.676470588,0.5,0.222222222,0.12345679,0.043209877,0.446143274,0.0,0.0,0.024908425,0.0,0.228887247,0.050473186,1.0,1.0,14.0,0.0,12032.0,162.0,10.0,0.09876543

In [12]:
data1==data2

False

In [4]:
#对数据进行缓存，同时统计数据样本的数目：
data.cache()
numData = data.count()
print (numData)

7395


在对数据集做进一步处理之前,我们发现数值数据中包含负的特征值。我们知道,朴素贝叶斯模型要求特征值非负,否则碰到负的特征值程序会抛出错误。因此,需要为朴素贝叶斯模型构建一份输入特征向量的数据,将负特征值设为0: 

In [12]:
nbdata = trimmed.map(lambda x : (x[-1],x[4:-1]))
nbdata1=nbdata.map(lambda kv:(kv[0].replace('\"',''),[0.0 if yy=='\"?\"' else yy.replace('\"','') for yy in kv[1]]))
nbdata2=nbdata1.map(lambda kv:(kv[0].replace("\"",""),[0.0 if yy =='?' else yy.replace("\"","") for yy in kv[1]]))
nbdata3=nbdata2.map(lambda kv:(int(kv[0]), [float(yy) for yy in kv[1]]))
nbdata4=nbdata3.map(lambda kv:(kv[0], [0.0 if yy<0 else yy for yy in kv[1]]))
nbdata =nbdata4.map(lambda kv:LabeledPoint(kv[0],Vectors.dense(kv[1])))
nbdata.take(2)

[LabeledPoint(0.0, [0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575]),
 LabeledPoint(1.0, [0.574147,3.677966102,0.50802139,0.288770053,0.213903743,0.144385027,0.468648998,0.0,0.0,0.098707403,0.0,0.203489628,0.088652482,1.0,1.0,40.0,0.0,4973.0,187.0,9.0,0.181818182,0.125448029])]

In [13]:
#导入相应的类
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.classification import SVMWithSGD
from pyspark.mllib.classification import NaiveBayes
from pyspark.mllib.tree import DecisionTree

numIteration = 10   #迭代次数
maxTreeDepth = 5    #树的深度

numClass = label.distinct().count()     #类别数
print ('类别数：',numClass)

#训练逻辑回归、支持向量机、朴素贝叶斯和决策树模型
lrModel = LogisticRegressionWithSGD.train(data, numIteration)
svmModel = SVMWithSGD.train(data, numIteration)
nbModel = NaiveBayes.train(nbdata)
dtModel = DecisionTree.trainClassifier(data,numClass,{},impurity='entropy', maxDepth=maxTreeDepth)
print ('逻辑回归模型参数：',lrModel)
print ('支持向量机模型参数：',svmModel)
print ('朴素贝叶斯模型参数：',nbModel)
print ('决策树模型参数：',dtModel)

类别数： 2


D:\anaconda1\lib\site-packages\pyspark\mllib\classification.py:313: UserWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "Deprecated in 2.0.0. Use ml.classification.LogisticRegression or "


逻辑回归模型参数： (weights=[-0.110216274454,-0.493200344739,-0.0712665620384,-0.0214744216778,0.00276706475384,0.00246385887598,-1.33300460292,0.0525232672351,0.0,-0.0320576776,-0.00653638798541,-0.0613702511674,-0.14975863133,-0.13648187383,-0.121161700009,-15.6451616669,-0.0177690355464,745.987958686,-7.73567729685,-1.38587998188,-0.0355600416613,-0.0352085128613], intercept=0.0)
支持向量机模型参数： (weights=[-0.122188386978,-0.527510758159,-0.0742371782434,-0.0206667449306,0.00546395033577,0.00409811283781,-1.54824523474,0.0607028905087,0.0,-0.037008323802,-0.007374037142,-0.067970375864,-0.172289581054,-0.148716595522,-0.129369384966,-18.0315472516,-0.0202704220321,1025.48043141,-5.05188911633,-1.54111193167,-0.038689478606,-0.0397619278886], intercept=0.0)
朴素贝叶斯模型参数： <pyspark.mllib.classification.NaiveBayesModel object at 0x000002753E7F17B8>
决策树模型参数： DecisionTreeModel classifier of depth 5 with 61 nodes


--------------------
4 使用分类模型 
--------------------

In [18]:
dataPoint = data.first()
prediction = lrModel.predict(dataPoint.features)
print ('预测的类别：',prediction)
print ('真实的类别：',int(dataPoint.label))

预测的类别： 1
真实的类别： 0


In [19]:
predictions = lrModel.predict(data.map(lambda x : x.features))
print ('前十个样本的预测标签：',predictions.take(10))
print ('前十个样本的真实标签：',data.map(lambda x : x.label).map(lambda x: int(x)).take(10))

前十个样本的预测标签： [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
前十个样本的真实标签： [0, 1, 1, 1, 0, 0, 1, 0, 1, 1]


-----------------------
5 评估分类模型的性能 
-----------------------

-----------------------------
5.1 预测的正确率和错误率
-----------------------------

In [20]:
#逻辑回归模型模型的预测正确数目
lrTotalCorrect = data.map(lambda point : 1 if(lrModel.predict(point.features)==point.label) else 0).sum()
#支持向量机模型的预测正确数目
svmTotalCorrect = data.map(lambda point : 1 if(svmModel.predict(point.features)==point.label) else 0).sum()
#朴素贝叶斯模型的预测正确数目
nbTotalCorrect = nbdata.map(lambda point : 1 if (nbModel.predict(point.features) == point.label) else 0).sum()

#决策树模型的预测正确数目
predictLabel= dtModel.predict(data.map(lambda point: point.features)).collect()
trueLabel = data.map(lambda point: point.label).collect()
dtTotalCorrect = sum([1.0 if prediction == trueLabel[i] else 0.0 for i, prediction in enumerate(predictLabel)])

#逻辑回归模型模型的预测正确率
lrAccuracy = lrTotalCorrect/(data.count()*1.0)
#支持向量机模型的预测正确率
svmAccuracy = svmTotalCorrect/(data.count()*1.0)
#朴素贝叶斯模型的预测正确率
nbAccuracy = nbTotalCorrect/(1.0*nbdata.count())
#决策树模型的预测正确率
dtAccuracy = dtTotalCorrect/(1.0*data.count())
print ('总共的样本数目: %s'%data.count())
print ('逻辑回归模型模型的预测正确率: %s'%lrAccuracy)
print ('支持向量机模型的预测正确率: %f'%svmAccuracy)
print ('朴素贝叶斯模型的预测正确率: %f'%nbAccuracy)
print ('决策树模型的预测正确率: %f'%dtAccuracy)

总共的样本数目: 7395
逻辑回归模型模型的预测正确率: 0.5146720757268425
支持向量机模型的预测正确率: 0.514672
朴素贝叶斯模型的预测正确率: 0.580392
决策树模型的预测正确率: 0.648276


-------------------------
5.2 准确率和召回率 
-------------------------

In [16]:
#MLlib内置了一系列方法用来计算二分类的PR和ROC曲线下的面积。
# 模型评价
from pyspark.mllib.evaluation import BinaryClassificationMetrics
all_models_metrics = []
for model in [lrModel, svmModel]:
    scoresAndLabels = data.map(lambda point:(model.predict(point.features), point.label)).collect()
    scoresAndLabels = [[float(i),j] for i,j in scoresAndLabels]
    
    rdd_scoresAndLabels = sc.parallelize(scoresAndLabels)   #将数据转换为rdd
    metrics = BinaryClassificationMetrics(rdd_scoresAndLabels)
    all_models_metrics.append((model.__class__.__name__, metrics.areaUnderROC, metrics.areaUnderPR))

for modelName, AUC, PR in all_models_metrics:
    print ('%s的AUC是%f,PR是%f'%(modelName, AUC, PR))


LogisticRegressionModel的AUC是0.501418,PR是0.756759
SVMModel的AUC是0.501418,PR是0.756759


In [22]:
#之前已经训练朴素贝叶斯模型并计算准确率,其中使用的数据集是nbData,这里使用同样的数据集进行模型评价。
predictionLabels = dtModel.predict(data.map(lambda point: point.features)).collect()
trueLabels = data.map(lambda point: point.label).collect()
scoresAndLabels = [[prediction, trueLabel] for prediction,trueLabel in zip(predictionLabels, trueLabels)]
scoresAndLabels = [[float(i),j] for i,j in scoresAndLabels]
rdd_scoresAndLabels = sc.parallelize(scoresAndLabels)

dt_metric = BinaryClassificationMetrics(rdd_scoresAndLabels)
print ('%s的AUC是%f,PR是%f'%(dtModel.__class__.__name__, dt_metric.areaUnderROC, dt_metric.areaUnderPR))

DecisionTreeModel的AUC是0.648837,PR是0.743081


-------------------------
6 改进模型性能以及参数调优 
-------------------------

----------------
6.1 特征标准化 
----------------

In [7]:

from pyspark.mllib.stat import MultivariateStatisticalSummary
from pyspark.mllib.linalg.distributed import RowMatrix

vectors = data.map(lambda point : point.features)
matrix = MultivariateStatisticalSummary(vectors)
matrix_mean = matrix.mean()
matrix_min = matrix.min()
print ('数据集每列的均值:',matrix_mean)
print ('数据集每列的最小值:',matrix_min)
print ('数据集每列的最大值:',matrix.max())
print ('数据集的样本数:',matrix.count())
print ('数据集每列的方差:',matrix.variance())



Exception ignored in: <bound method JavaModelWrapper.__del__ of <pyspark.mllib.stat._statistics.MultivariateStatisticalSummary object at 0x000002753E75D198>>
Traceback (most recent call last):
  File "D:\anaconda1\lib\site-packages\pyspark\mllib\common.py", line 142, in __del__
    self._sc._gateway.detach(self._java_model)
  File "D:\anaconda1\lib\site-packages\py4j\java_gateway.py", line 1870, in detach
    java_object._detach()
AttributeError: 'PipelinedRDD' object has no attribute '_detach'


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 28, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "E:\Spark\spark-2.1.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 174, in main
  File "E:\Spark\spark-2.1.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 169, in process
  File "E:\Spark\spark-2.1.0-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "D:\anaconda1\lib\site-packages\pyspark\rdd.py", line 833, in func
    yield reduce(f, iterator, initial)
TypeError: unorderable types: DenseVector() < DenseVector()

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scal



观察前面对均值和方差的输出,可以清晰发现第二个特征的方差和均值比其他的都要高(你会发现一些其他特征也有类似的结果,而且有些特征更加极端)。因为我们的数据在原始形式下,确切地说并不符合标准的高斯分布。为使数据更符合模型的假设,可以对每个特征进行标准化,使得每个特征是0均值和单位标准差。具体做法是对每个特征值减去列的均值,然后除以列的标准差以进行缩放:                  






                                              (x–μ) / sqrt(variance) 





使用Spark的StandardScaler中的方法方便地完成这个任务。 首先，传入两个参数,一个表示是否从数据中减去均值,另一个表示是否应用标准差缩放。这样使得StandardScaler和我们的输入向量相符。最后,将输入向量传到转换函数,并且返回归一化的向量。具体实现代码如下,我们使用map函数来保留数据集的标签: 





In [9]:
from pyspark.mllib.feature import StandardScalerModel,StandardScaler

scaler = StandardScaler(withMean=True, withStd=True).fit(vectors)
labels = data.map(lambda point: point.label)
features = data.map(lambda point: point.features)
print ('原始数据:',features.take(3))
scaled_data = labels.zip(scaler.transform(features))
scaled_data = scaled_data.map(lambda kv: LabeledPoint(kv[0],kv[1]))
print ('规范化之后的数据:',scaled_data.map(lambda point: point.features).take(3))

原始数据: [DenseVector([0.7891, 2.0556, 0.6765, 0.2059, 0.0471, 0.0235, 0.4438, 0.0, 0.0, 0.0908, 0.0, 0.2458, 0.0039, 1.0, 1.0, 24.0, 0.0, 5424.0, 170.0, 8.0, 0.1529, 0.0791]), DenseVector([0.5741, 3.678, 0.508, 0.2888, 0.2139, 0.1444, 0.4686, 0.0, 0.0, 0.0987, 0.0, 0.2035, 0.0887, 1.0, 1.0, 40.0, 0.0, 4973.0, 187.0, 9.0, 0.1818, 0.1254]), DenseVector([0.9965, 2.3829, 0.562, 0.3217, 0.1202, 0.0426, 0.5254, 0.0, 0.0, 0.0724, 0.0, 0.2264, 0.1205, 1.0, 1.0, 55.0, 0.0, 2240.0, 258.0, 11.0, 0.1667, 0.0576])]
规范化之后的数据: [DenseVector([1.1376, -0.0819, 1.0251, -0.0559, -0.4689, -0.3543, -0.3175, 0.3385, 0.0, 0.8288, -0.1473, 0.2296, -0.1416, 0.7902, 0.7172, -0.298, -0.2035, -0.033, -0.0488, 0.9401, -0.1087, -0.2788]), DenseVector([0.4887, 0.1063, 0.1959, 0.509, 1.2695, 1.3097, -0.3132, 0.3385, 0.0, 1.0202, -0.1473, -0.5771, -0.0975, 0.7902, 0.7172, 0.4866, -0.2035, -0.0838, 0.0459, 1.2494, 0.0489, 0.3058]), DenseVector([1.7637, -0.044, 0.4617, 0.7334, 0.2927, -0.0912, -0.3032, 0.3385, 0.0, 0.3867,

In [14]:
#现在我们使用标准化的数据重新训练模型。这里只训练逻辑回归(因为决策树和朴素贝叶斯不受特征标准化的影响),并说明特征标准化的影响: 
lrModel_scaled = LogisticRegressionWithSGD.train(scaled_data,numIteration)
lrPredictionLabel = lrModel_scaled.predict(scaled_data.map(lambda point: point.features))
trueLabel = scaled_data.map(lambda point: point.label)
lrTotalCorrect_scaled = sum([1.0 if prediction==label else 0.0 
                             for prediction, label in zip(lrPredictionLabel.collect(),trueLabel.collect())])/scaled_data.count()
print ('特征规范化之后逻辑回归预测的正确率:',lrTotalCorrect_scaled)

D:\anaconda1\lib\site-packages\pyspark\mllib\classification.py:313: UserWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "Deprecated in 2.0.0. Use ml.classification.LogisticRegression or "


特征规范化之后逻辑回归预测的正确率: 0.6209601081812035


In [17]:
all_models_metrics =[]
for model in [lrModel_scaled]:
    scoresAndLabels = scaled_data.map(lambda point:(model.predict(point.features),point.label)).collect()
    scoresAndLabels = [(float(i),j) for (i,j) in scoresAndLabels]
    scoresAndLabels_rdd = sc.parallelize(scoresAndLabels)
    metrics = BinaryClassificationMetrics(scoresAndLabels_rdd)
    all_models_metrics.append((model.__class__.__name__,metrics.areaUnderROC, metrics.areaUnderPR))
for model_name, AUC, PR in all_models_metrics:
    print ('%s的AUC是%f,PR是%f'%(model_name, AUC, PR))

LogisticRegressionModel的AUC是0.620190,PR是0.727701


---------------
6.2 其他特征 
---------------

In [20]:
category_dict = {}
categories = records.map(lambda x: x[3]).distinct().zipWithIndex().collect()
print (categories)
for  (x,y) in [(key.replace('\"','') ,val) for (key, val) in categories]:
    category_dict[x] = y

print ('类别的编码:',category_dict)

num_categories = len(category_dict)
print ('类别数:',num_categories)

[('"recreation"', 0), ('"culture_politics"', 1), ('"law_crime"', 2), ('"weather"', 3), ('"religion"', 4), ('"computer_internet"', 5), ('"science_technology"', 6), ('"unknown"', 7), ('"health"', 8), ('"sports"', 9), ('"?"', 10), ('"arts_entertainment"', 11), ('"gaming"', 12), ('"business"', 13)]
类别的编码: {'sports': 9, 'religion': 4, 'business': 13, 'culture_politics': 1, 'unknown': 7, 'arts_entertainment': 11, 'recreation': 0, 'law_crime': 2, 'health': 8, '?': 10, 'weather': 3, 'science_technology': 6, 'gaming': 12, 'computer_internet': 5}
类别数: 14


In [21]:
otherdata = trimmed.map(lambda x : (x[-1],x[4:-1]))
otherdata1=otherdata.map(lambda kv:(kv[0].replace('\"',''),[0.0 if yy=='\"?\"' else yy.replace('\"','') for yy in kv[1]]))
otherdata2=otherdata1.map(lambda kv:(kv[0].replace("\"",""),[0.0 if yy =='?' else yy.replace("\"","") for yy in kv[1]]))
otherdata3=otherdata2.map(lambda kv:(int(kv[0]), [float(yy) for yy in kv[1]]))
otherdata=otherdata3.map(lambda kv:LabeledPoint(kv[0],Vectors.dense(kv[1])))
otherdata.take(5)

[LabeledPoint(0.0, [0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575]),
 LabeledPoint(1.0, [0.574147,3.677966102,0.50802139,0.288770053,0.213903743,0.144385027,0.468648998,0.0,0.0,0.098707403,0.0,0.203489628,0.088652482,1.0,1.0,40.0,0.0,4973.0,187.0,9.0,0.181818182,0.125448029]),
 LabeledPoint(1.0, [0.996526,2.382882883,0.562015504,0.321705426,0.120155039,0.042635659,0.525448029,0.0,0.0,0.072447859,0.0,0.22640177,0.120535714,1.0,1.0,55.0,0.0,2240.0,258.0,11.0,0.166666667,0.057613169]),
 LabeledPoint(1.0, [0.801248,1.543103448,0.4,0.1,0.016666667,0.0,0.480724749,0.0,0.0,0.095860566,0.0,0.265655744,0.035343035,1.0,0.0,24.0,0.0,2737.0,120.0,5.0,0.041666667,0.100858369]),
 LabeledPoint(0.0, [0.719157,2.676470588,0.5,0.222222222,0.12345679,0.043209877,0.446143274,0.0,0.0,0.024908425,0.0,0.228887247,0.050473186,1.0,1.0,14.0,0.0,12032.0,162.0,10.0,0.09876543

In [22]:
#将类别特征和数值特征合并
def build_feature(x):
    import numpy as np
    numeric_feature = [0.0 if yy=='?' else yy for yy in [y.replace('\"','') for y in x[4:-1]]]  #数值特征
    category_feature = np.zeros(num_categories)              #数值特征
    category_index = category_dict[x[3].replace('\"','')]    
    category_feature[category_index] = 1                     #类别特征
    label = x[-1].replace('\"','')                           #样本标签
    feature = LabeledPoint(label, Vectors.dense(list(category_feature)+numeric_feature))   #合并类别特征和数值特征
    return feature

category_data = trimmed.map(lambda x:build_feature(x))
category_data.take(5)

[LabeledPoint(0.0, [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.789131,2.055555556,0.676470588,0.205882353,0.047058824,0.023529412,0.443783175,0.0,0.0,0.09077381,0.0,0.245831182,0.003883495,1.0,1.0,24.0,0.0,5424.0,170.0,8.0,0.152941176,0.079129575]),
 LabeledPoint(1.0, [1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.574147,3.677966102,0.50802139,0.288770053,0.213903743,0.144385027,0.468648998,0.0,0.0,0.098707403,0.0,0.203489628,0.088652482,1.0,1.0,40.0,0.0,4973.0,187.0,9.0,0.181818182,0.125448029]),
 LabeledPoint(1.0, [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.996526,2.382882883,0.562015504,0.321705426,0.120155039,0.042635659,0.525448029,0.0,0.0,0.072447859,0.0,0.22640177,0.120535714,1.0,1.0,55.0,0.0,2240.0,258.0,11.0,0.166666667,0.057613169]),
 LabeledPoint(1.0, [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.801248,1.543103448,0.4,0.1,0.016666667,0.0,0.480724749,0.0,0.0,0.095860566,0.0,0.265655744,0.035343035,1.0,0.0,24.0,0.0,2737.0

In [24]:
#同样,因为我们的原始数据没有标准化,所以在训练这个扩展数据集之前,应该使用同样的StandardScaler方法对其进行标准化转换: 
category_labels = category_data.map(lambda point: point.label)
category_features = category_data.map(lambda point: point.features)
scaler2 = StandardScaler(withMean=True, withStd=True).fit(category_features)
print ('规范化之前的数据集特征:',category_features.take(5))
scaled_category_data = category_labels.zip(scaler2.transform(category_features))
scaled_category_data = scaled_category_data.map(lambda kv: LabeledPoint(kv[0],kv[1]))
print ('规范化之后的数据集特征:',scaled_category_data.map(lambda point: point.features).take(5))

规范化之前的数据集特征: [DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.7891, 2.0556, 0.6765, 0.2059, 0.0471, 0.0235, 0.4438, 0.0, 0.0, 0.0908, 0.0, 0.2458, 0.0039, 1.0, 1.0, 24.0, 0.0, 5424.0, 170.0, 8.0, 0.1529, 0.0791]), DenseVector([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5741, 3.678, 0.508, 0.2888, 0.2139, 0.1444, 0.4686, 0.0, 0.0, 0.0987, 0.0, 0.2035, 0.0887, 1.0, 1.0, 40.0, 0.0, 4973.0, 187.0, 9.0, 0.1818, 0.1254]), DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9965, 2.3829, 0.562, 0.3217, 0.1202, 0.0426, 0.5254, 0.0, 0.0, 0.0724, 0.0, 0.2264, 0.1205, 1.0, 1.0, 55.0, 0.0, 2240.0, 258.0, 11.0, 0.1667, 0.0576]), DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8012, 1.5431, 0.4, 0.1, 0.0167, 0.0, 0.4807, 0.0, 0.0, 0.0959, 0.0, 0.2657, 0.0353, 1.0, 0.0, 24.0, 0.0, 2737.0, 120.0, 5.0, 0.0417, 0.1009]), DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [25]:
#计算模型准确率
lrModel_category_scaled = LogisticRegressionWithSGD.train(scaled_category_data, numIteration)
lr_totalCorrect_category_scaled = scaled_category_data.map(lambda point : 1 if(lrModel_category_scaled.predict(point.features)==point.label) else 0).sum()
lr_accuracy_category_scaled = lr_totalCorrect_category_scaled/(1.0*data.count())
print ('逻辑回归模型的准确率 : %f'%lr_accuracy_category_scaled)

D:\anaconda1\lib\site-packages\pyspark\mllib\classification.py:313: UserWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "Deprecated in 2.0.0. Use ml.classification.LogisticRegression or "


逻辑回归模型的准确率 : 0.665720


In [26]:
#计算模型的AUC和PR
all_models_metrics =[]
for model in [lrModel_category_scaled]:
    scoresAndLabels = scaled_category_data.map(lambda point:(model.predict(point.features),point.label)).collect()
    scoresAndLabels = [(float(i),j) for (i,j) in scoresAndLabels]
    scoresAndLabels_rdd = sc.parallelize(scoresAndLabels)
    metrics = BinaryClassificationMetrics(scoresAndLabels_rdd)
    all_models_metrics.append((model.__class__.__name__,metrics.areaUnderROC, metrics.areaUnderPR))

for model_name, AUC, PR in all_models_metrics:
    print ('%s的AUC是%f,PR是%f'%(model_name, AUC, PR))

LogisticRegressionModel的AUC是0.665475,PR是0.757982


---------------------
6.3 使用正确的数据格式 
---------------------



模型性能的另外一个关键部分是对每个模型使用正确的数据格式。前面对数值向量应用朴素贝叶斯模型得到了非常差的结果，这是模型本身的缺陷还是因为其他原因造成的呢？在这里,我们知道MLlib实现了多项式模型,并且该模型可以处理计数形式的数据。这包括二元表示的类型特征(比如前面提到的1-of-k表示)或者频率数据(比如一个文档中单词出现的频率)。我开始时使用的数值特征并不符合假定的输入分布,所以模型性能不好也并不是意料之外。 




为了更好的说明问题，本文仅仅使用类型特征,而1-of-k编码的类型特征更符合朴素贝叶斯模型,我们用如下代码构建数据集: 


In [27]:
category_nbdata = scaled_category_data
category_nbdata_nonegative = category_nbdata.map(lambda point: (point.label,point.features)).map(lambda kv: (kv[0],[0.0 if yy<0.0 else yy for yy in kv[1]])).map(lambda kv:LabeledPoint(kv[0],Vectors.dense(kv[1])))

In [29]:
category_nbdata_nonegative.take(2)

[LabeledPoint(0.0, [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.72073665645,1.1376473365,0.0,1.02513981289,0.0,0.0,0.0,0.0,0.33845079824,0.0,0.828822173315,0.0,0.229639823578,0.0,0.790238049918,0.717194729453,0.0,0.0,0.0,0.0,0.940069975117,0.0,0.0]),
 LabeledPoint(1.0, [2.23973405107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.488685990417,0.106283637051,0.195885662909,0.508986806825,1.26946916328,1.30971389846,0.0,0.33845079824,0.0,1.02024383053,0.0,0.0,0.0,0.790238049918,0.717194729453,0.486582251769,0.0,0.0,0.0459442290216,1.24936955983,0.0488534204631,0.305780221901])]

In [28]:
#计算准确率
nb_category_model = NaiveBayes.train(category_nbdata_nonegative)
nb_category_total_correct = category_nbdata_nonegative.map(lambda point:1 if (nb_category_model.predict(point.features)==point.label) else 0).sum()

nb_category_accuracy = nb_category_total_correct/(1.0*category_nbdata_nonegative.count())
print ('朴素贝叶斯模型的准确率:%f'%nb_category_accuracy)

朴素贝叶斯模型的准确率:0.652738


In [30]:
all_models_metrics =[]
for model in [nb_category_model]:
    scoresAndLabels = category_nbdata_nonegative.map(lambda point:(model.predict(point.features),point.label)).collect()
    scoresAndLabels = [(float(i),j) for (i,j) in scoresAndLabels]
    scoresAndLabels_rdd = sc.parallelize(scoresAndLabels)
    metrics = BinaryClassificationMetrics(scoresAndLabels_rdd)
    all_models_metrics.append((model.__class__.__name__,metrics.areaUnderROC, metrics.areaUnderPR))

for model_name, AUC, PR in all_models_metrics:
    print ('%s的AUC是%f,PR是%f'%(model_name, AUC, PR))



NaiveBayesModel的AUC是0.651468,PR是0.752038


从上述结果可知,使用格式正确的输入数据后 ,朴素贝叶斯的准确率从58%提高到了65.27%。 

--------------------
6.4 模型参数调优 
--------------------

---------------------------
6.4.1 逻辑回归
---------------------------

前几节展示了模型性能的影响因素:特征提取、特征选择、输入数据的格式和模型对数据分布的假设。但是到目前为止,我们对模型参数的讨论只是一笔带过,而实际上它对于模型性能影响很大。

MLlib默认的train方法对每个模型的参数都使用默认值。接下来让我们深入了解一下这些参数。 下面就演示一下在Spark Python中我们如何进行这部分的工作。


In [31]:
def train_with_params(input_data, model, reg_param, num_iter, step_size):
    model = model.train(input_data, iterations=num_iter, regParam=reg_param, step=step_size)
    return model
def create_metrics(tag, input_data, model):
    scoresAndLabels_rdd = input_data.map(lambda x: (model.predict(x.features)*1.0,x.label*1.0))
    metrics = BinaryClassificationMetrics(scoresAndLabels_rdd)
    return tag, metrics.areaUnderROC, metrics.areaUnderPR

In [32]:
#对参数迭代次数进行调优
for iteration_num in [1,5,10,50]:
    model = train_with_params(scaled_category_data, LogisticRegressionWithSGD, 0.0, iteration_num, 1.0)
    label, ROC, PR = create_metrics('%d iterations'%iteration_num,scaled_category_data,model)
    print ('%s,ROC = %f,PR=%f'%(label,ROC,PR))

D:\anaconda1\lib\site-packages\pyspark\mllib\classification.py:313: UserWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "Deprecated in 2.0.0. Use ml.classification.LogisticRegression or "


1 iterations,ROC = 0.649520,PR=0.745886
5 iterations,ROC = 0.666161,PR=0.758022
10 iterations,ROC = 0.665483,PR=0.757964
50 iterations,ROC = 0.668143,PR=0.760930


从上述结果可知，迭代次数达到一定值之后，再增加迭代次数对结果影响已经较小了。 

对参数步长进行调优：











在SGD中,在训练每个样本并更新模型的权重向量时,步长用来控制算法在最陡的梯度方向上应该前进多远。较大的步长收敛较快,但是步长太大可能导致收敛到局部最优解。下面计算不同步长的影响：


In [33]:
#对参数步长次数进行调优
for step_size in [0.001, 0.01, 0.1, 1.0, 10.0]:
    model = train_with_params(scaled_category_data, LogisticRegressionWithSGD, 0.0, 10, step_size)
    label, ROC, PR = create_metrics('%2.3f 的步长'%step_size,scaled_category_data,model)
    print ('%s,ROC = %f,PR=%f'%(label,ROC,PR))

D:\anaconda1\lib\site-packages\pyspark\mllib\classification.py:313: UserWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "Deprecated in 2.0.0. Use ml.classification.LogisticRegression or "


0.001 的步长,ROC = 0.649659,PR=0.745974
0.010 的步长,ROC = 0.649644,PR=0.746017
0.100 的步长,ROC = 0.655211,PR=0.750008
1.000 的步长,ROC = 0.665483,PR=0.757964
10.000 的步长,ROC = 0.619228,PR=0.725713


从上述结果可以看出，步长增长过大对模型性能有负面影响。

对参数正则化系数进行调优： 







正则化的具体做法是在损失函数中添加一项关于模型权重向量的函数,从而会使损失增加。正则化在现实中几乎是必须的,当特征维度高于训练样本时(此时变量相关需要学习的权重数量也非常大)尤其重要。

当正则化不存在或者非常低时,模型容易过拟合。而且大多数模型在没有正则化的情况会在训练数据上过拟合。过拟合也是交叉验证技术使用的关键原因,交叉验证会在后面详细介绍。

相反,虽然正则化可以得到一个简单模型,但正则化太高可能导致模型欠拟合,从而使模型性能变得很糟糕。 




MLlib中可用的正则化形式有如下几个。

 SimpleUpdater:相当于没有正则化,是逻辑回归的默认配置。SquaredL2Updater:这个正则项基于权重向量的L2正则化,是SVM模型的默认值。

 L1Updater:这个正则项基于权重向量的L1正则化,会导致得到一个稀疏的权重向量(不重要的权重的值接近0)。 


In [34]:
#对参数正则化系数进行调优
for reg_param in [0.001, 0.01, 0.1, 1.0, 10.0]:
    model = train_with_params(scaled_category_data, LogisticRegressionWithSGD, reg_param, 10, 0.01)
    label, ROC, PR = create_metrics('正则化系数%2.3f '%reg_param,scaled_category_data,model)
    print ('%s,ROC = %f,PR=%f'%(label,ROC,PR))

D:\anaconda1\lib\site-packages\pyspark\mllib\classification.py:313: UserWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "Deprecated in 2.0.0. Use ml.classification.LogisticRegression or "


正则化系数0.001 ,ROC = 0.649644,PR=0.746017
正则化系数0.010 ,ROC = 0.649644,PR=0.746017
正则化系数0.100 ,ROC = 0.649644,PR=0.746017
正则化系数1.000 ,ROC = 0.649644,PR=0.746017
正则化系数10.000 ,ROC = 0.648979,PR=0.745492


------------------
6.4.2 决策树
------------------



决策树模型在一开始使用原始数据做训练时获得了最好的性能。当时设置了参数maxDepth用来控制决策树的最大深度,进而控制模型的复杂度。而树的深度越大,得到的模型越复杂,但有能力更好地拟合数据。对于分类问题,我们需要为决策树模型选择以下两种不纯度度量方式:Gini或者Entropy。 








对树的深度和不纯度调优 


In [35]:
def train_with_params_dt(input_data, impurity, maxTreeDepth):
    dt_model = DecisionTree.trainClassifier(input_data,numClass,{},impurity, maxDepth=maxTreeDepth)
    return dt_model
def create_metrics_dt(tag, input_data, model):
    predictLabel= model.predict(input_data.map(lambda point: point.features)).collect()
    trueLabel = input_data.map(lambda point: point.label).collect()
    scoresAndLabels = [(predictLabel[i],true_val) for i, true_val in enumerate(trueLabel)]
    scoresAndLabels_rdd = sc.parallelize(scoresAndLabels)
    scoresAndLabels_rdd = scoresAndLabels_rdd.map(lambda kv: (float(kv[0]),float(kv[1])))
    dt_metrics = BinaryClassificationMetrics(scoresAndLabels_rdd)
    return tag,dt_metrics.areaUnderROC,dt_metrics.areaUnderPR

In [36]:
for depth in [1,2,3,4,5,10,20]:
    for im in ['entropy','gini']:
        model=train_with_params_dt(data,im,depth)
        tag, ROC, PR = create_metrics_dt('impurity: %s, %d maxTreeDepth:'%(im,depth),data,model)
        print ('%s, AUC=%f, PR=%f'%(tag,ROC,PR))



impurity: entropy, 1 maxTreeDepth:, AUC=0.593268, PR=0.749004
impurity: gini, 1 maxTreeDepth:, AUC=0.593268, PR=0.749004
impurity: entropy, 2 maxTreeDepth:, AUC=0.616839, PR=0.725164
impurity: gini, 2 maxTreeDepth:, AUC=0.616839, PR=0.725164
impurity: entropy, 3 maxTreeDepth:, AUC=0.626070, PR=0.751813
impurity: gini, 3 maxTreeDepth:, AUC=0.626070, PR=0.751813
impurity: entropy, 4 maxTreeDepth:, AUC=0.636333, PR=0.749393
impurity: gini, 4 maxTreeDepth:, AUC=0.636333, PR=0.749393
impurity: entropy, 5 maxTreeDepth:, AUC=0.648837, PR=0.743081
impurity: gini, 5 maxTreeDepth:, AUC=0.648916, PR=0.742894
impurity: entropy, 10 maxTreeDepth:, AUC=0.762552, PR=0.829623
impurity: gini, 10 maxTreeDepth:, AUC=0.783709, PR=0.843469
impurity: entropy, 20 maxTreeDepth:, AUC=0.984537, PR=0.988522
impurity: gini, 20 maxTreeDepth:, AUC=0.988707, PR=0.991328


从结果中可以看出,提高树的深度可以得到更精确的模型(这和预期一致,因为模型在更大的树深度下会变得更加复杂)。然而树的深度越大,模型对训练数据过拟合程度越严重。另外，两种不纯度方法对性能的影响差异不大。

----------------------
6.4.3 朴素贝叶斯
----------------------

最后,让我们看看lamda参数对朴素贝叶斯模型的影响。该参数可以控制相加式平滑(additive smoothing),解决数据中某个类别和某个特征值的组合没有同时出现的问题 。

In [37]:
def train_with_params_nb(input_data, lambda_para):
    nb_model = NaiveBayes.train(input_data,lambda_para)
    return nb_model
def create_metrics_nb(tag, nbdata, model):
    scoresAndLabels = nbdata.map(lambda point:(float(model.predict(point.features)),point.label))
    nb_metrics = BinaryClassificationMetrics(scoresAndLabels)
    return tag,nb_metrics.areaUnderROC,nb_metrics.areaUnderPR

In [38]:
for lambda_para in [0.001, 0.01, 0.1, 1.0, 10.0]:
    model=train_with_params_nb(nbdata,lambda_para)
    tag, ROC, PR = create_metrics_dt('lambda=%f' %lambda_para,nbdata,model)
    print ('%s, AUC=%f, PR=%f'%(tag,ROC,PR))

lambda=0.001000, AUC=0.583697, PR=0.680961
lambda=0.010000, AUC=0.583697, PR=0.680961
lambda=0.100000, AUC=0.583697, PR=0.680961
lambda=1.000000, AUC=0.583559, PR=0.680851
lambda=10.000000, AUC=0.583412, PR=0.680762


从结果中可以看出lambda的值对性能没有影响,由此可见数据中某个特征和某个类别的组合不存在时不是问题。 

-------------------------
6.4.4 交叉验证
-------------------------






交叉验证是实际机器学习中的关键部分，同时在多模型选择和参数调优中占有很重的地位。



交叉验证的目的是测试模型在未知数据上的性能。不知道训练的模型在预测新数据时的性能,而直接放在实际数据(比如运行的系统)中进行评估是很危险的做法。正如前面提到的正则化实验中,我们的模型可能在训练数据中已经过拟合了,于是在未被训练的新数据中预测性能会很差。



交叉验证让我们使用一部分数据训练模型,将另外一部分用来评估模型性能。如果模型在训练以外的新数据中进行了测试,我们便可以由此估计模型对新数据的泛化能力。我们把数据划分为训练和测试数据,实现一个简单的交叉验证过程。我们将数据分为两个不重叠的数据集。第一个数据集用来训练,称为训练集。第二个数据集称为测试集或者保留集,用来评估模型在给定评测方法下的性能。实际中常用的划分方法包括:50/50、60/40、80/20等,只要训练模型的数据量不太小就行(通常,实际使用至少50%的数据用于训练)。 




在建模过程中,通常会创建三个数据集:训练集、评估集(类似上述测试集用于模型参数的调优,比如lambda和步长)和测试集(不用于模型的训练和参数调优,只用于估计模型在新数据中性能)。 




本文将数据集分成60%的训练集和40%的测试集(为了方便解释,我们在代码中使用一个固定的随机种子123来保证每次实验能得到相同的结果): 


In [40]:
train_test_split = scaled_category_data.randomSplit([0.6,0.4],123)
train = train_test_split[0]
test = train_test_split[1]
for reg_para in [0.0, 0.001, 0.0025, 0.005, 0.01]:
    model = train_with_params(train, LogisticRegressionWithSGD, 0.0, 1.0, reg_para)
    label, ROC, PR = create_metrics('%f regularization parameter'%reg_para,test,model)
    print ('%s,AUC = %f,PR=%f'%(label,ROC,PR))

D:\anaconda1\lib\site-packages\pyspark\mllib\classification.py:313: UserWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "Deprecated in 2.0.0. Use ml.classification.LogisticRegression or "


IllegalArgumentException: 'requirement failed: Initial step size must be positive but got 0.0'